In [3]:
import tensorflow as tf
import os

# Étape 1 : Définir le chemin du modèle local
model_path = r"C:\Users\marti\Desktop\Nouveau dossier\Homework_9\model_2024_hairstyle.keras"

# Vérification que le fichier existe
try:
    print(f"Chargement du modèle depuis le fichier local : {model_path}")
    model_keras = tf.keras.models.load_model(model_path)
    print("Modèle chargé avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle : {e}")
    exit()  # Quittez si le chargement échoue

# Étape 2 : Conversion en modèle TensorFlow Lite
print("Conversion en modèle TensorFlow Lite...")
converter = tf.lite.TFLiteConverter.from_keras_model(model_keras)
tflite_model = converter.convert()

# Étape 3 : Sauvegarder le modèle converti dans le même répertoire
output_dir = os.path.dirname(model_path)  # Obtenir le répertoire du modèle original
output_path = os.path.join(output_dir, "model_2024_hairstyle.tflite")

try:
    with open(output_path, "wb") as f:
        f.write(tflite_model)
    print(f"Modèle TensorFlow Lite sauvegardé avec succès : {output_path}")
except Exception as e:
    print(f"Erreur lors de la sauvegarde du modèle : {e}")
    exit()

# Étape 4 : Afficher la taille du modèle TF-Lite
model_size_in_mb = len(tflite_model) / (1024 * 1024)
print(f"Taille du modèle TensorFlow Lite converti : {model_size_in_mb:.2f} MB")


Chargement du modèle depuis le fichier local : C:\Users\marti\Desktop\Nouveau dossier\Homework_9\model_2024_hairstyle.keras
Modèle chargé avec succès.
Conversion en modèle TensorFlow Lite...
INFO:tensorflow:Assets written to: C:\Users\marti\AppData\Local\Temp\tmpjzlteynq\assets


INFO:tensorflow:Assets written to: C:\Users\marti\AppData\Local\Temp\tmpjzlteynq\assets


Saved artifact at 'C:\Users\marti\AppData\Local\Temp\tmpjzlteynq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2094987922944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2094523337888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2093964045200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2093964043616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2093962791744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2094523425088: TensorSpec(shape=(), dtype=tf.resource, name=None)
Modèle TensorFlow Lite sauvegardé avec succès : C:\Users\marti\Desktop\Nouveau dossier\Homework_9\model_2024_hairstyle.tflite
Taille du modèle TensorFlow Lite converti : 76.58 MB


In [5]:

# Charger le modèle TF-Lite
interpreter = tf.lite.Interpreter(model_path=r"C:\Users\marti\Desktop\Nouveau dossier\Homework_9\model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

# Récupérer les indices d'entrée et de sortie
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Afficher les détails de sortie
print(f"Output index: {output_details[0]['index']}")


Output index: 13


In [19]:
from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np

# Fonction pour télécharger une image
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

# Fonction pour redimensionner l'image
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# Télécharger et redimensionner l'image
image_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(image_url)
img = prepare_image(img, (200, 200))  # Taille cible

# Convertir en tableau numpy
img_array = np.array(img) / 255.0  # Normalisation des pixels entre 0 et 1
print(f"First pixel (R channel): {img_array[0, 0, 0]:.2f}")


First pixel (R channel): 0.24


In [20]:
print(f"Forme initiale de l'image : {img_array.shape}")


Forme initiale de l'image : (200, 200, 3)


In [21]:
img_array = np.squeeze(img_array)  # Supprime les dimensions de taille 1


In [22]:
img_array = np.expand_dims(img_array, axis=0)


In [23]:
img_array = img_array.astype(np.float32)


In [24]:
print(f"Forme corrigée de l'image : {img_array.shape}")
print(f"Forme attendue par le modèle : {input_details[0]['shape']}")


Forme corrigée de l'image : (1, 200, 200, 3)
Forme attendue par le modèle : [  1 200 200   3]


In [25]:
# Exécuter le modèle
interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()

# Obtenir les résultats
output_data = interpreter.get_tensor(output_details[0]['index'])
print(f"Model output: {output_data}")

Model output: [[0.8937741]]


In [1]:
!pip install requests


In [2]:
!pip show requests


Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: c:\users\marti\desktop\code\venv\lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: tensorflow-intel
